# Decision Tree algorithm

Decision Tree là thuật toán supervised, có thể giải quyết cả bài toán regression và classification.

## Giới thiệu về thuật toán Decision Tree

Một thuật toán Machine Learning thường sẽ có 2 bước:
1. Training: Từ dữ liệu thuật toán sẽ học ra model.
2. Prediction: Dùng model học được từ bước trên dự đoán các giá trị mới.

Bước training ở thuật toán Decision Tree, sẽ xây ra một cây quyết định.
Ví dụ, như với dữ liệu Titatic, thuật toán Decision Tree sẽ học ra model dạng cây như thế này

![titanic_decision_tree](./imgs/titanic.png)

Thông tin Title được lấy ra từ trường Name. Sau đó trường Title, Sex được chuyển về dạng số

In [3]:
title_mapping   = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
sex_mapping     = {'female': 0, 'male': 1}

Sau đó ở bước dự đoán, thuật toán sẽ dựa vào thông tin của hành khách và đi theo các điều kiện của cây từ trên xuống dưới để cho ra
dự đoán xem người đó sống hay chết. Ví dụ với thông tin khách hàng thế này:

In [4]:
import pandas as pd
data = pd.read_csv('../data/train.csv')
data.head(1)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S


Từ trường Name, mình sẽ lấy được Title là "Mr" rồi chuyển về dạng số là 1.
1. Title = 1 <= 1.5, điều kiện đúng nên kiểm tra tiếp điều kiện ở node con ở dưới bên trái.
2. Has_Cabin = 0 <= 0.5, điều kiện đúng nên kiểm tra tiếp điều kiện ở node con ở dưới bên trái (trường Cabin là NaN nên has_cabin thành 0).
3. Pclass = 3 >= 1.5, điều kiện sai nên xuống node con bên phải, node này là node lá có kết quả dự đoán luôn chứ không cần kiểm tra điều kiện nữa.
4. Lấy thông tin từ node lá, dự đoán class = Died.

![titanic_decision_tree_predict](./imgs/titanic_predict.png)

Ở cây điều kiện này mình thấy 2 kiểu node:
1. Node có điều kiện kiểm tra, mình gọi là node điều kiện. Các node điều kiện đều có 2 node con ở dưới.
2. Node lá, không có điều kiện mà có kết quả dự đoán. Các node lá không có node con.

Bây giờ thì mình biết có cây quyết định thì sẽ dự đoán 1 giá trị mới như thế nào rồi. Vấn đề bây giờ là làm thế nào để xây cây quyết định.

## Xây cây quyết định

Bài toán là giờ mình có dữ liệu, làm thế nào để xây ra cây quyết định.

Giả sử mình có bài toán loại 2 lớp và mỗi dữ liệu có 2 thuộc tính là $x_1$ và $x_2$. Dữ liệu của mình khi vẽ biểu đồ scatter lên sẽ như thế này.

![visualize](./imgs/visualize.png)

Với dữ liệu này, nếu yêu cầu mọi người dùng giấy bút vẽ cây quyết định mọi người sẽ làm thế nào?

Mọi người nhìn điều điện đầu tiên trên cùng $x_1 > 5$, giống như một đường phân chia, chia dữ liệu làm 2 phần, 1 phần thỏa mãn điều kiện và 1 phần không thỏa mãn điều kiện.

![visualize](./imgs/cond_1.png)

Mình thấy nếu $x_1 > 5$ đúng thì tất cả các dữ liệu thuộc lớp 1, thế nên mình sẽ dùng lớp lá để dự đoán đây là lớp 1 luôn. Ngược lại thì mình thấy dữ liệu có cả lớp 1 và lớp 0, nên mình tiếp tục thêm nhánh điều kiện $x_2 > 4$

![visualize](./imgs/cond_2.png)

Nếu điều kiện $x_2 > 4$ đúng thì mình thấy các dữ liệu thuộc lớp 1, ngược lại các dữ liệu thuộc lớp 0. Do đó 2 node con của node điều kiện trên đều là node lá để cho ra kết quả dự đoán.

Cuối cùng mình sẽ được cây quyết định như thế này.

![visualize](./imgs/data_predict_1.png)

Vậy tiêu chí gì để mình tìm được điều kiện đầu tiên? tại sao lại là $x_1$ và tại sao lại là 5 mà không phải là một số khác? 
Nếu mọi người để ý ở trên thì mình sẽ tạo điều kiện để tách dữ liệu thành 2 phần mà dữ liệu mỗi phần có tính phân tách hơn dữ liệu ban đầu. Ví dụ: điều kiện $x_1 > 5$, tại nhánh đúng thì tất cả các phần tử đều thuộc lớp 1.
 - Thế điều kiện $x_1 > 8 $ cũng chia nhánh đúng thành toàn lớp 1 sao không chọn? vì nhánh đúng ở điều kiện $x_1 > 5$ chứa nhiều phần tử lớp 1 hơn và tổng quát hơn nhán đúng của $x_1 > 8$
 - Còn điều kiện $x_1 > 2 $ thì cả 2 nhánh đều chứa dữ liệu của cả lớp 0 và lớp 1.
 
 Mình biết tiêu chí để chọn rồi, nhưng tiêu chí mình vừa đề ra dựa vào mắt và cảm quan, máy tính thì cần số liệu để đánh giá để so sánh các điều kiện phân tách. Các chỉ số để đánh giá ra đời, bao gồm: **gini index** và **entropy**.